# Automatic Identification of Internal Waves Competition
+ This notebook belongs ot the competition Automatic Identification of Internal Waves
+ This notebook was created to test the competition data.
+ The performance results provide a baseline for the competition.
+ The metric used to compare models is Kaggle's Roc Auc Score: Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores. 
  + [Link to Kaggle metrics](https://www.kaggle.com/competitions/automatic-identification-of-internal-waves/host/evaluation)
  + [link to source definition](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from skimage.io import imread
from skimage.transform import resize


: 

In [ ]:
# Set random seed for reproducibility
np.random.seed(1)

In [ ]:
# Define paths
train_dir = '/Users/jpinelo/Dropbox/JP_Lab/AIRCentre/2-Projects/54-Internal_waves/Data/images_train/'
test_dir = '/Users/jpinelo/Dropbox/JP_Lab/AIRCentre/2-Projects/54-Internal_waves/Data/images_test/'
train_csv = '/Users/jpinelo/Dropbox/JP_Lab/AIRCentre/2-Projects/54-Internal_waves/Data/train.csv'
test_csv = '/Users/jpinelo/Dropbox/JP_Lab/AIRCentre/2-Projects/54-Internal_waves/Data/test.csv'
solution_csv = '/Users/jpinelo/Dropbox/JP_Lab/AIRCentre/2-Projects/54-Internal_waves/Data/solution.csv'

In [ ]:
# Load CSV files
train_df = pd.read_csv(train_csv)
train_df['id'] = train_df['id'].astype(str) + '.png'
test_df = pd.read_csv(test_csv)
test_df['id'] = test_df['id'].astype(str) + '.png'
solution_df = pd.read_csv(solution_csv)
solution_df['id'] = solution_df['id'].astype(str) + '.png'

In [ ]:
# Define function to load images
def load_images(directory, df):
    images = []
    for img_name in df['id']:
        img_path = os.path.join(directory, img_name)
        img = imread(img_path)
        img_resized = resize(img, (50, 50, 4))  # Resize to smaller dimension for faster processing
        images.append(img_resized.flatten())  # Flatten the image
    return np.array(images)

In [ ]:
# Load and preprocess the images
X_train = load_images(train_dir, train_df)
X_test = load_images(test_dir, test_df)
y_train = train_df['ground_truth'].values

In [ ]:
# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Create and train the model
model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=5000, random_state=42)
model.fit(X_train_scaled, y_train)

In [ ]:
# Use the model to make predictions on the validation set
val_predictions = model.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
# Make probability predictions on the test set
test_probabilities = model.predict_proba(X_test_scaled)[:, 1]  # Get probabilities for class 1

In [ ]:
# Add probabilities to the test DataFrame
test_df['predicted_probability'] = test_probabilities

: 

In [ ]:
# Compare predictions with the solution provided (test set)
merged_df = pd.merge(test_df, solution_df, on='id')

In [ ]:
# Calculate performance metrics
accuracy = accuracy_score(merged_df['ground_truth'], (merged_df['predicted_probability'] > 0.5).astype(int))
precision = precision_score(merged_df['ground_truth'], (merged_df['predicted_probability'] > 0.5).astype(int))
recall = recall_score(merged_df['ground_truth'], (merged_df['predicted_probability'] > 0.5).astype(int))
f1 = f1_score(merged_df['ground_truth'], (merged_df['predicted_probability'] > 0.5).astype(int))

In [ ]:
# Compute ROC AUC
roc_auc = roc_auc_score(merged_df['ground_truth'], merged_df['predicted_probability'])

In [ ]:
# Print performance metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

In [ ]:
# Save the results to a CSV file
#test_df[['id', 'predicted_probability']].to_csv('/Users/jpinelo/Dropbox/JP_Lab/AIRCentre/2-Projects/54-Internal_waves/Data/submission-baseline.csv', index=False)